obectives of this program

1. the context of coalescence( background )

2. My concept and plan about this research

3. future plan(next step)

equations

inputs and outputs
why do we use this method

Import the packages

In [1]:
   using Pkg
   Pkg.activate("../packages")
   using Distributions
   using MiCRM

  Activating project at `~/Documents/digital_microbiome/MiCRM/packages`


Set system size and leakage rate

In [2]:
   N,M,leakage = 100,50,0.3

(100, 50, 0.3)

N is the number of species in a community, M is the number of resource types, leakage is the overall leakage rate

Make uptake matrix out of dirichlet distribution

In [3]:
   u =  MiCRM.Parameters.modular_uptake(M,N; N_modules = 2, s_ratio = 10.0)

50×100 Matrix{Float64}:
 0.0228498    0.0279291    0.028028     …  0.00149532   0.00298185
 0.0164318    0.0288344    0.0262614       0.00233634   0.00291566
 0.0305011    0.00871968   0.0041449       0.00155349   0.000308966
 0.00203741   0.0236238    0.0325241       0.003839     0.00171899
 0.018145     0.0182143    0.00980698      0.00224639   0.00211286
 0.017946     0.0292746    0.0101066    …  0.00245168   0.00109866
 0.0111957    0.00599084   0.0140818       0.00321148   0.00188048
 0.00397256   0.0241991    0.00801332      0.00112261   0.00218671
 0.0315154    0.0263172    0.0101395       0.0026384    0.000291593
 0.0329976    0.00661551   0.00906015      0.000604361  0.00240581
 0.00576665   0.0282021    0.0165121    …  0.00279692   0.00186496
 0.0177809    0.026757     0.0237751       1.95755e-5   0.00145403
 0.000882859  0.0108269    0.00854039      0.00323617   0.00140859
 ⋮                                      ⋱               
 0.00271767   0.00349249   0.000984726     0.0

<span style="color:gray">MiCRM.Parameters.modular_uptake</span> helps generate an uptake matrix using a Dirichlet distribution such that the uptake of all consumers sums to 1.
The number of modules determines how many groups of resources the consumers are specialised over. For example if N_modules = 2 then the resources will be split into two groups with half the consumers specialising on one and half on the other.

The degree of specialisation is determined by the s_ratio value. This controls the relative value of the dirchlet α parameters which determine how the probabiltiy density function is distributed over the different resources. Specialisation is obtained by setting the α values for resources that specialists consume to higher values meaning they have a higher probablity of a larger value. When s_ratio = 1 the proabailtiy is uniform and all resources are equally likely to be consumed. When s_ratio >1 then consumers are more likely to consume resources within thier module.

Define other terms

In [4]:
   #cost term
   m = ones(N)

   #resource inflow + outflow
   ρ,ω = ones(M),ones(M)

([1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0  …  1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0  …  1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0])

m is a loss term for the the consumers; ρ and ω are inflow rate and outflow rate for the resource respectively.

Make leakage matrix out of dirichlet distribution and make it modular

In [5]:
   l = MiCRM.Parameters.modular_leakage(M; N_modules = 2, s_ratio = 10.0, λ = leakage)

50×50 Matrix{Float64}:
 0.00236767  0.00707186   0.00482691   …  0.010215    0.00825083  0.00129627
 0.00225891  0.010467     0.0112036       0.00742818  0.0121214   0.00818068
 0.00552296  0.0056489    0.00957735      0.00801503  0.00890253  0.007585
 0.00371305  0.00233981   0.0102374       0.0035955   0.00761503  0.0129087
 0.00323117  0.0098897    0.00289703      0.00557908  0.00525277  0.000686345
 0.0102434   0.00997235   0.0070545    …  0.00471792  0.0033593   0.00120622
 0.0120573   0.00760947   0.00493816      0.00820794  0.00506066  0.00437889
 0.00721244  0.00999924   0.000681733     0.00644288  0.0123163   0.00200691
 0.00305751  0.00415367   0.00779058      0.00170265  0.0100492   0.00413012
 0.00938162  0.00719782   0.00875381      0.00443969  0.01089     0.00809999
 0.00705124  0.00167254   0.00642315   …  0.00116745  0.00277325  0.00667154
 0.00193507  0.0077338    0.00836205      0.00322352  0.014806    0.000837355
 0.00523006  0.00745684   0.0102029       0.00431155  

<span style="color:gray">`MiCRM.Parameters.modular_leakage`</span> generates a modular leakage matrix with a directional leakage structure. The matrix is generated using a Dirichlet distribution such that the leakage of all resources sums to λ.

The number of modules determines how many groups of resources they are split into. For example if N_modules = 5 then the resources will be split into five groups with the first group of resources tending to leak to the second, the second to the third and so on.

The degree of to which resources leak in this constrained way (verses randomly across all resources) is determined by the s_ratio value. This controls the relative value of the dirchlet α parameters which determine how the probabiltiy density function is distributed over the different resources. A greater probabiltiy of leakage from one resource to another is obtained by setting the α values for resources to higher values. When s_ratio = 1 the proabailtiy is uniform and all resources are equally likely to be leaked to each other. When s_ratio >1 then resources are more likely to leak to resources thier own or the next module in the sequence.

Set original state of the system and solve it

In [6]:
   #define parameters
   param = MiCRM.Parameters.generate_params(N, M;  u = u, m = m, ρ = ρ, ω = ω, l = l, λ = leakage)
   
   #original state
   x0 = ones(N+M)
   #time span
   tspan = (0.0, 10.0)

   #define problem
   using DifferentialEquations
   prob = ODEProblem(MiCRM.Simulations.dx!, x0, tspan, param)
   

ODEProblem with uType Vector{Float64} and tType Float64. In-place: true
timespan: (0.0, 10.0)
u0: 150-element Vector{Float64}:
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 ⋮
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0

create an ODEProblem object which defines the problem for the ODE solver and then solve it with the aptly named solve function.
To define the ODEProblem we need to specify the initial state of the system as well as the timespan

In [7]:
   sol = solve(prob, Tsit5())

retcode: Success
Interpolation: specialized 4th order "free" interpolation
t: 17-element Vector{Float64}:
  0.0
  0.08349282713382426
  0.21793651628353833
  0.3778294892447818
  0.5869204587569905
  0.8410546468729772
  1.1602406529574516
  1.5511013101274185
  2.0294609084124686
  2.5987665113973404
  3.2720407119533235
  4.063519181677232
  4.997140691603409
  6.103851758938936
  7.422459171604166
  8.967548970268178
 10.0
u: 17-element Vector{Vector{Float64}}:
 [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0  …  1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
 [0.9722227059326333, 0.972007295112164, 0.9722097547397758, 0.972020887630998, 0.9721967272335128, 0.972009159460745, 0.971871064699027, 0.9720970886253382, 0.9721417656599199, 0.9721958521717302  …  0.8649448893959677, 0.8770805271015476, 0.9223172670890349, 0.9312862710033691, 0.8974895990578442, 0.888226246495523, 0.881162082707563, 0.8901085119879929, 0.8751996101634885, 0.9262852237207305]
 [0.9191193221812958, 0.917

In this case, we use Tsit5 as a solver, which is widely applicable for non-stiff problems.
If need more information about the solver choice, refer to https://docs.sciml.ai/DiffEqDocs/stable/solvers/ode_solve/

Calculate key properties

In [8]:
   # jacobian of system from the solution object
   J = MiCRM.Analysis.get_jac(sol)
   # generate purbulence matrix
   pur = rand(size(J, 1))
   # the instantaneous rate of growth of the perturbation pur at time t.
   t = 5
   MiCRM.Analysis.get_Rins(J, pur, t)
   # Determine the stability a system given its jaccobian by testing if the real part of the leading eigenvalue is positive.
   MiCRM.Analysis.get_stability(J)
   #test whether a system is reactive to the perturbation 
   MiCRM.Analysis.get_reactivity(J,pur)
   #get the rate of return of the system from perturbation
   MiCRM.Analysis.get_return_rate(J)

-0.3184907555897299

visualization